<a href="https://colab.research.google.com/github/yoshimethod/bioinformatics/blob/main/bioinformatics_pr4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchmetrics==0.7.1
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.2/397.2 kB 5.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
import os
import urllib.request
import numpy as np
import pandas as pd
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional
import torch.optim
from sklearn.metrics import accuracy_score
from torchmetrics.functional import precision_recall
import matplotlib
import matplotlib.pyplot as plt
import Bio
from Bio.Seq import Seq

In [3]:
normal_rna =Seq('GCATGCTGCATGCATGATGCCATGCATGCTGCATGCTGCATG')
tumor_rna  =Seq('GCATGCTGCATGCATGATGCCACGCATGCTGCATGCTGCATG')

print('野生型RNA配列:..' + str(normal_rna) + "..")
print('腫瘍特異的RNA配列:..' + str(tumor_rna) + "..")

print('野生型タンパク質配列:..' + str(normal_rna.translate()) +"..")
print('腫瘍特異的タンパク質配列:..' + str(tumor_rna.translate()) + "..")

野生型RNA配列:..GCATGCTGCATGCATGATGCCATGCATGCTGCATGCTGCATG..
腫瘍特異的RNA配列:..GCATGCTGCATGCATGATGCCACGCATGCTGCATGCTGCATG..
野生型タンパク質配列:..ACCMHDAMHAACCM..
腫瘍特異的タンパク質配列:..ACCMHDATHAACCM..
